# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_2.json`):

In [2]:
{
  "name": "Example LCIA Method",
  "version": "1.0",
  "description": "Example LCIA method for greenhouse gas emissions",
  "unit": "kg CO2e",
  "exchanges": [
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "location": "CH"
      },
      "value": 1.0
    },
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "location": "FR"
      },
      "value": 2.0
    },
    {
      "supplier": {
        "name": "Carbon dioxide",
        "operator": "startswith",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "location": "RER"
      },
      "value": 3.0
    }
  ]
}



{'name': 'Example LCIA Method',
 'version': '1.0',
 'description': 'Example LCIA method for greenhouse gas emissions',
 'unit': 'kg CO2e',
 'exchanges': [{'supplier': {'name': 'Carbon dioxide',
    'operator': 'startswith',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'location': 'CH'},
   'value': 1.0},
  {'supplier': {'name': 'Carbon dioxide',
    'operator': 'startswith',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'location': 'FR'},
   'value': 2.0},
  {'supplier': {'name': 'Carbon dioxide',
    'operator': 'startswith',
    'matrix': 'biosphere'},
   'consumer': {'matrix': 'technosphere', 'location': 'RER'},
   'value': 3.0}]}

In [3]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10.1-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (megajoule, RU, None)

In [4]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/Users/romain/micromamba/envs/ab/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 33.73it/s]

1210 processed biosphere edges
0 processed technosphere edges


0.00016281503408902597

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [5]:
df = LCA.generate_cf_table()

In [6]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of waste wood, untreated, municipal ...","waste wood, untreated",CH,1.910804e-07,1.0,1.910804e-07
1,"Carbon dioxide, non-fossil","(air, urban air close to ground)","biomethane, low pressure burned in micro gas t...","heat, central or small-scale, other than natur...",CH,6.295395e-21,1.0,6.295395e-21
2,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of digester sludge, municipal incine...","heat, for reuse in municipal waste incineratio...",CH,7.323961e-10,1.0,7.323961e-10
3,"Carbon dioxide, non-fossil","(air, urban air close to ground)","lath, hardwood, raw, kiln drying to u=20%","sawnwood, lath, hardwood, raw, dried (u=20%)",CH,2.838967e-11,1.0,2.838967e-11
4,"Carbon dioxide, non-fossil","(air, urban air close to ground)",beet sugar production,sugar beet pulp,CH,1.623526e-30,1.0,1.623526e-30
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, in air","(natural resource, in air)",rye production,straw,RER,2.353219e-09,3.0,7.059656e-09
1206,"Carbon dioxide, in air","(natural resource, in air)",rye production,rye grain,RER,9.056163e-11,3.0,2.716849e-10
1207,"Carbon dioxide, in air","(natural resource, in air)",anthraquinone production,anthraquinone,RER,2.776034e-12,3.0,8.328101e-12
1208,"Carbon dioxide, in air","(natural resource, in air)",latex production,latex,RER,1.003052e-15,3.0,3.009155e-15


As expected, only CH, FR and RER-based consumers have been considered in the calculation.

In [7]:
df.groupby("consumer location")["CF"].mean()

consumer location
CH     1.0
FR     2.0
RER    3.0
Name: CF, dtype: float64

But `statistics()` tells us that many exchanges have been ignored.

In [8]:
LCA.statistics()

+----------------------+--------------------------------------------+
|       Activity       | heat and power co-generation, natural gas, |
|                      | conventional power plant, 100MW electrical |
|     Method name      |             ('some', 'method')             |
|      Data file       |               lcia_example_2               |
|    CFs in method     |                     3                      |
|       CFs used       |                     3                      |
|  Exc. characterized  |                    1210                    |
| Exc. uncharacterized |                   334699                   |
+----------------------+--------------------------------------------+


We can extend the application of CFs to other exchanges, using the helping functions: `map_aggregate_locations`, `map_disaggregate_location`, `map_dynamic_locations` and `map_remaining_locations_to_global`.

* `map_aggregate_locations` maps exchanges with aggregate locations (e.g., "RER", RLA", etc.) to weighted-average CFs.
* `map_disaggregate_location` maps exchanges with locations that belong to aggregate locations ("CA-QC") to CFs of aggregate locations (e.g., "CA").
* `map_dynamic_locations` maps exchanges with dynamic locations (e.g., "RoW", "RoW") to CFs calculated as the weighted-average of all locations minus those excluded by dynamic locations' perimeter (e.g., "GLO minus CH").
* `map_remaining_locations_to_global` maps exchanges with remaining locations to the global CFs.

In [9]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_contained_locations()
LCA.map_remaining_locations_to_global()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/Users/romain/micromamba/envs/ab/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 128.22it/s]

1210 processed biosphere edges
0 processed technosphere edges
Handling static regions...



Processing static groups (pass 1): 0it [00:00, ?it/s]
Processing remaining static edges (pass 2): 100%|█| 96785/96785 [00:00<00:00, 53
Processing static groups (pass 1): 0it [00:00, ?it/s]
Processing remaining static edges (pass 2): 0it [00:00, ?it/s]


...

Processing remaining dynamic edges (pass 2): 100%|█| 84848/84848 [00:00<00:00, 5


2713 processed biosphere edges
0 processed technosphere edges
Handling contained locations...
2713 processed biosphere edges
0 processed technosphere edges
Handling remaining exchanges...


Processing remaining global edges (pass 2): 100%|█| 333196/333196 [00:00<00:00, 


6462 processed biosphere edges
0 processed technosphere edges


0.07194876358318945

In [10]:
LCA.statistics()

+----------------------+--------------------------------------------+
|       Activity       | heat and power co-generation, natural gas, |
|                      | conventional power plant, 100MW electrical |
|     Method name      |             ('some', 'method')             |
|      Data file       |               lcia_example_2               |
|    CFs in method     |                     3                      |
|       CFs used       |                    5255                    |
|  Exc. characterized  |                    6462                    |
| Exc. uncharacterized |                   329447                   |
+----------------------+--------------------------------------------+


In [11]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

In [12]:
df = LCA.generate_cf_table()

In [13]:
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of waste wood, untreated, municipal ...","waste wood, untreated",CH,1.910804e-07,1.000000,1.910804e-07
1,"Carbon dioxide, non-fossil","(air, urban air close to ground)","biomethane, low pressure burned in micro gas t...","heat, central or small-scale, other than natur...",CH,6.295395e-21,1.000000,6.295395e-21
2,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of digester sludge, municipal incine...","heat, for reuse in municipal waste incineratio...",CH,7.323961e-10,1.000000,7.323961e-10
3,"Carbon dioxide, non-fossil","(air, urban air close to ground)","lath, hardwood, raw, kiln drying to u=20%","sawnwood, lath, hardwood, raw, dried (u=20%)",CH,2.838967e-11,1.000000,2.838967e-11
4,"Carbon dioxide, non-fossil","(air, urban air close to ground)",beet sugar production,sugar beet pulp,CH,1.623526e-30,1.000000,1.623526e-30
...,...,...,...,...,...,...,...,...
6457,"Carbon dioxide, non-fossil, resource correction","(natural resource, in air)","sulfate pulp production, from eucalyptus, blea...","sulfate pulp, bleached",RoW,2.903644e-11,1.881782,5.464025e-11
6458,"Carbon dioxide, non-fossil","(air, urban air close to ground)","heat and power co-generation, wood chips, 6667 kW","heat, district or industrial, other than natur...",EE,3.120166e-08,1.881782,5.871472e-08
6459,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","electricity production, hydro, pumped storage","electricity, high voltage",LU,1.147676e-09,1.881782,2.159676e-09
6460,"Carbon dioxide, fossil","(air,)","electricity production, oil","electricity, high voltage",CN-JS,1.281791e-10,1.881782,2.412050e-10


We can check the weights used:

In [14]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

By default, if weights are not provided in the consumer sections of LCIA file, population is used. The other option is GDP. To be specified when calling EdgeLICA().

And we can see the weighted-average CFs used:

In [15]:
df.groupby("consumer location")["CF"].mean()

consumer location
AE        1.881782
AR        1.881782
AT        1.881782
AU        1.881782
AU-NSW    1.881782
            ...   
VE        1.881782
VN        1.881782
WEU       1.881782
ZA        1.881782
ZM        1.881782
Name: CF, Length: 227, dtype: float64

We have pre-generated LCIA files for specific methods (AWARE, ImpactWorld+, etc.).
You can get a list like so:

In [16]:
get_available_methods()

[('AWARE 2.0', 'Country', 'all', 'yearly'),
 ('AWARE 2.0', 'Country', 'irri', 'yearly'),
 ('AWARE 2.0', 'Country', 'non', 'irri', 'yearly'),
 ('AWARE 2.0', 'Country', 'unspecified', 'yearly'),
 ('GeoPolRisk', '2024'),
 ('GeoPolRisk', 'paired', '2024'),
 ('ImpactWorld+ 2.1', 'Freshwater acidification', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater acidification', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity, long term', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity, long term', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity, short term', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater ecotoxicity, short term', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Land occupation, biodiversity', 'damage'),
 ('ImpactWorld+ 2.1', 'La

Check the other notebooks to see how to use them.